In [ ]:
%pip install -q -r requirements.txt

In [ ]:

import functions.core.common_functions as cf
import importlib

importlib.reload(cf)

config_path = "/home/developer/keys/project-keys/colab-settings.yaml"
# service_account_secret_name = "SA_ADHOC_BILLING"

# # Initialize client manager
# veil_client_manager = cf.GCPClientManager(config_path, service_account_secret_name)

# # Get clients
# veil_bigquery_client = veil_client_manager.get_bigquery_client()
# veil_storage_client = veil_client_manager.get_storage_client()
# veil_salesforce_client = veil_client_manager.get_salesforce_client()

# print("Veil clients initialized successfully!")

# service_account_secret_name = "SA_N90_CORE_APPS"

# # Initialize client manager
# n90_client_manager = cf.GCPClientManager(config_path, service_account_secret_name)

# # Get clients
# n90_bigquery_client = n90_client_manager.get_bigquery_client()
# n90_storage_client = n90_client_manager.get_storage_client()

# print("N90 clients initialized successfully!")

In [ ]:
# import importlib
import functions.core.core_functions as core_functions
import functions.pyarrow_functions as pyarrow_functions
import dask
import dask.dataframe as dd
from dask.dataframe.utils import assert_eq
import pandas as pd
import numpy as np
import pandas_gbq
import json
import gc
import cudf
import os
import yaml
from concurrent.futures import ProcessPoolExecutor
from functools import partial
# import uu

dask.config.set({"dataframe.backend": "cudf"})

importlib.reload(core_functions)
importlib.reload(pyarrow_functions)

In [ ]:
table_schema = yaml.safe_load(open('table-schemas.yaml'))

detections_python = table_schema['detections_python_schema']
detections_python

n90_schema_dict = {item['name']: item['type'] for item in detections_python}
veil_schema_dict = {item['name']: item['type'] for item in detections_python if 'geo_' not in item['name']}
final_detections_cols = []
for item in detections_python:
    final_detections_cols.append(item['name'])
    # print(f"{item['name']} = {item['type']}")

In [ ]:
resp = {}
resp = core_functions.initialize_clients(service_account_secret_name='SA_ADHOC_BILLING')
resp2 = core_functions.initialize_clients(service_account_secret_name='SA_N90_CORE_APPS')

config = resp.get('config')
bigquery_client = resp.get('clients').get('bigquery_client')
n90_bigquery_client = resp2.get('clients').get('bigquery_client')
storage_client = resp.get('clients').get('storage_client')
sf_client = resp.get('clients').get('sf_client')
veil_billing = resp.get('config').get('veil_billing')
veil_vars = resp.get('config').get('veil_billing').get('vars')
# print(veil_billing)
sfdc_adv_account_cols = veil_billing.get('vars').get('sfdc_adv_account_cols')
sfdc_rate_card_cols = veil_billing.get('vars').get('sfdc_rate_card_cols')
unknown_dma_overrides = config.get('national_dma_overrides_to_us_national')

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_ADHOC_BILLING')
broadcast_cal_sql = f"""
    SELECT id as bcw_id, bcw_index, bcm_index, bcw_start_date, bcw_start_datetime_match, bcw_end_date, bcw_end_datetime_match FROM `adhoc-billing.avs_billing_process.lu_broadcast_week`
"""
broadcast_cal_df = core_functions.fetch_gbq_data(query=broadcast_cal_sql, bigquery_client=bigquery_client)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_N90_CORE_APPS')

In [ ]:
broadcast_cal_df

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_N90_CORE_APPS')
geo_zip_sql = f"""
    SELECT *
    from `next90-core-applications.next90_analytics.geos` WHERE geo_type = 'zip'
    AND geo_country in ('United States', 'Canada')
"""
geo_zip_df = core_functions.fix_df_dtypes(core_functions.fetch_gbq_data(query=geo_zip_sql, bigquery_client=n90_bigquery_client))
geo_zip_df['geo_latitude'] = geo_zip_df['geo_latitude'].astype('Float64')
geo_zip_df['geo_longitude'] = geo_zip_df['geo_longitude'].astype('Float64')


geo_dma_sql = f"""
    SELECT *
    from `next90-core-applications.next90_analytics.geos` WHERE geo_type = 'dma'
    AND geo_country in ('United States', 'Canada')
"""
geo_dma_df = core_functions.fix_df_dtypes(core_functions.fetch_gbq_data(query=geo_dma_sql, bigquery_client=n90_bigquery_client))
geo_dma_df['geo_latitude'] = geo_dma_df['geo_latitude'].astype('Float64')
geo_dma_df['geo_longitude'] = geo_dma_df['geo_longitude'].astype('Float64')



# int_cols = ['geo_neustar_id','geo_us_msa_id', 'geo_us_county_fips_id','geo_ca_cma_id']
# for col in int_cols:
#     geo_zip_df[col] = geo_df[col].fillna(-1).astype('Int64')

In [ ]:
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_N90_CORE_APPS')
# broadcast_cal_sql = f"""
#     SELECT id as bcw_id, bcw_index, bcm_index, bcw_start_date, bcw_end_date FROM `next90-core-applications.n90_data_lake.lu_broadcast_week`
# """
# broadcast_cal_df = core_functions.fetch_gbq_data(query=broadcast_cal_sql, bigquery_client=n90_bigquery_client)

In [ ]:
geo_dma_df.dtypes

In [ ]:
data_start_date = '2023-01-01'
data_end_date = '2024-01-01'

process_sql = f"""
    SELECT distinct EXTRACT(YEAR FROM created_time) as year, EXTRACT(MONTH FROM created_time) as month, EXTRACT(DAY FROM created_time) as  day
 from `next90-core-applications.omniData.activity_sessions`
    WHERE created_time >= '{data_start_date}' and created_time < '{data_end_date}'
    order by year, month, day
"""

process_df = core_functions.fetch_gbq_data(process_sql, n90_bigquery_client)
process_df

In [ ]:
process_days = process_df.to_dict(orient='records')
process_days
for record in process_days:
    print(f'year: {record["year"]}, month: {record["month"]}, day: {record["day"]}')

In [ ]:
import concurrent.futures
import pandas as pd
import os
import gc
from datetime import datetime
from google.cloud import bigquery, bigquery_storage

def process_activity_sessions_segments(_df, _bcc_df, geo_z_df, geo_d_df, media='DIGITAL'):
    print('starting process_activity_sessions_segments')
    print('adding broadcast cal details')
    print(f'length of df: {len(_df)}')
    
    # Ensure bcw_monday_match and created_time are datetime
    _df['activity_date_time'] = pd.to_datetime(_df['created_time']).dt.strftime('%Y-%m-%d %H:%M:%S')
    _df['bcw_monday_match'] = pd.to_datetime(_df['bcw_monday_match'])
    _df['click_view_gclid'] = _df['gclid'].astype('string')

    # Find the broadcast week boundaries
    min_date = _df['bcw_monday_match'].min()
    max_date = _df['bcw_monday_match'].max()

    # Ensure the Mondays before and after are captured
    min_monday = min_date - pd.Timedelta(days=7 if min_date.weekday() == 0 else min_date.weekday())
    max_monday = max_date + pd.Timedelta(days=7 - max_date.weekday())

    # Generate a full list of Mondays between min and max
    monday_match_dates = pd.date_range(start=min_monday, end=max_monday, freq='7D').date.tolist()

    # Filter _bcc_df to include only valid broadcast weeks
    _bcc_df['bcw_start_date'] = pd.to_datetime(_bcc_df['bcw_start_date'])
    ref_df = _bcc_df.loc[
        _bcc_df['bcw_start_date'].dt.date.isin(monday_match_dates)
    ].drop_duplicates().reset_index(drop=True)

    # Merge _df and _bcc_df based on date conditions
    merged_df = pd.merge(_df, ref_df, how='cross')  # Cartesian join (safer with cross in pandas 1.2+)
    print(f"Rows after Cartesian join: {len(merged_df)}")
    merged_df = merged_df.loc[
        (merged_df['activity_date_time'] >= merged_df['bcw_start_datetime_match']) &
        (merged_df['activity_date_time'] < merged_df['bcw_end_datetime_match'])
    ].reset_index(drop=True)
    print(f"Rows after activity date filter: {len(merged_df)}")

    # Final processing
    merged_df['bcw_year'] = merged_df['bcw_index'].astype(str).str[:4].astype('Int64')
    merged_df['bcm_index'] = merged_df['bcm_index'].astype('Float64')
    merged_df['bcw_index'] = merged_df['bcw_index'].astype('Float64')


    _df = merged_df.copy().reset_index(drop=True)
    del merged_df
    # del merged_df2
    gc.collect()
    _df['_YEAR'] = _df['created_time'].dt.year.astype('Int64')
    _df['_MONTH'] = _df['created_time'].dt.month.astype('Int64')
    _df['_WEEK_WITHIN_MONTH'] = _df['week_within_month'].astype('Int64')
    _df['_DAY'] = _df['created_time'].dt.day.astype('Int64')
    print('adding geos')
    activity_sessions_with_geos_df = _df.merge(geo_z_df, how='left', left_on='zip_code', right_on='geo_location')
    del _df
    gc.collect()
    activity_sessions_with_geos_df['neustar_dma_id'] = activity_sessions_with_geos_df['neustar_dma_id'].astype('string')
    activity_sessions_with_geos_df.loc[activity_sessions_with_geos_df['neustar_country'].isin(['us', 'ca']) & activity_sessions_with_geos_df['geo_country'].isnull()] = activity_sessions_with_geos_df.loc[activity_sessions_with_geos_df['neustar_country'].isin(['us', 'ca']) & activity_sessions_with_geos_df['geo_country'].isnull()].merge(geo_d_df, how='left', left_on='neustar_dma_id', right_on='geo_location')
    activity_sessions_with_geos_df['neustar_dma_id'] = activity_sessions_with_geos_df['neustar_dma_id'].astype('Int64')
    # activity_sessions_with_geos_df.sort_values(by=['activity_session_id', 'created_time'], ascending=[True, False] ,inplace=True)
    activity_sessions_with_geos_df = activity_sessions_with_geos_df.dropna(subset=['activity_session_id'])
    _df = activity_sessions_with_geos_df.copy().reset_index(drop=True)
    # .drop_duplicates(subset=['activity_session_id'], keep='first').sort_values(by='created_time').copy().reset_index(drop=True)
    del activity_sessions_with_geos_df
    gc.collect()
    
    print('adding segments')
    _df['segments_date'] = pd.to_datetime(_df['created_time'])
    _df['segments_day_of_week'] = pd.to_datetime(_df['segments_date']).dt.tz_localize(None).dt.day_name().astype('string')
    _df['segments_media'] =  media
        
    _df['segments_month_label'] = pd.to_datetime(_df['segments_date']).dt.tz_localize(None).dt.to_period('M').astype('string')
    _df['segments_quarter_label'] = pd.to_datetime(_df['segments_date']).dt.tz_localize(None).dt.to_period('Q').astype('string')
    _df['segments_week_label'] = pd.to_datetime(_df['segments_date']).dt.tz_localize(None).dt.to_period('W').astype('string')

    # Convert timestamp to periods and then use start_time to get the first day of the period
    _df['segments_month'] = pd.to_datetime(_df['segments_date']).dt.tz_localize(None).dt.to_period('M').dt.start_time.dt.date.astype('string')
    _df['segments_quarter'] = pd.to_datetime(_df['segments_date']).dt.tz_localize(None).dt.to_period('Q').dt.start_time.dt.date.astype('string')
    _df['segments_week'] = pd.to_datetime(_df['segments_date']).dt.tz_localize(None).dt.to_period('W').dt.start_time.dt.date.astype('string')

    # Year can remain as a period or also be converted similarly if needed
    _df['segments_year'] = pd.to_datetime(_df['segments_date']).dt.tz_localize(None).dt.to_period('Y').astype('string')
    _df['segments_broadcast_year'] = _df['bcw_year'].astype('Int64')
    _df['segments_broadcast_month_index'] = _df['bcm_index'].astype('Float64')
    _df['segments_broadcast_week_index'] = _df['bcw_index'].astype('Float64')
    # _df['_YEAR'] = _df['segments_date'].dt.year.astype('Int64')
    # _df['_MONTH'] = _df['segments_date'].dt.month.astype('Int64')
    # _df['_DAY'] = _df['segments_date'].dt.day.astype('Int64')
    _df['segments_date'] = _df['segments_date'].dt.date.astype('string')
    _df['session_timestamp'] = _df['created_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    _df['created_time'] = _df['created_time'].dt.strftime('%Y-%m-%d %H:%M:%S')

    _df['bcw_start_date'] = _df['bcw_start_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    _df['bcw_end_date'] = _df['bcw_end_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    print('finished process_activity_sessions_segments')
    

    return _df

def process_single_day(record, broadcast_cal_df, geo_zip_df, geo_dma_df, n90_bucket, n90_output_prefix, s4_bucket, s4_output_prefix, partition_cols, n90_storage_options, s4_storage_options):
    try:
        year = record['year']
        month = record['month']
        day = record['day']
        print(f'Processing year: {year}, month: {month}, day: {day}')

        activity_sessions_sql = f"""
            WITH temp_activity_sessions AS (
            SELECT DISTINCT s.*, CAST((EXTRACT(DAY FROM created_time) - 1) / 7 + 1 AS INT64) AS week_within_month,
            row_number() over (partition by s.id order by s.created_time desc) as row_num
            FROM `next90-core-applications.omniData.temp_activity_sessions`  s
            WHERE EXTRACT(YEAR FROM created_time) = {year}
            AND EXTRACT(MONTH FROM created_time) = {month}
            AND EXTRACT(DAY FROM created_time) = {day}
            )
            SELECT * FROM temp_activity_sessions
            where row_num = 1
        """

        # Load data
        print('Loading activity sessions')
        # try:
        #     activity_sessions_df = core_functions.fetch_gbq_data(activity_sessions_sql, n90_bigquery_client)
        # except Exception as e:
        #     print(f"Error loading activity sessions with core_functions, reading directly: {e}")
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_N90_CORE_APPS')
        activity_sessions_df = bigquery.Client().query(activity_sessions_sql).result().to_dataframe()
        # activity_sessions_df = bigquery.Client(credentials=config.get('SA_N90_CORE_APPS')).query(activity_sessions_sql).result().to_dataframe()
        
        activity_sessions_df.rename(columns={'id': 'activity_session_id'}, inplace=True)
        activity_sessions_df['lat'] = activity_sessions_df['lat'].astype('Float64')
        activity_sessions_df['lon'] = activity_sessions_df['lon'].astype('Float64')

        print('Processing with function process_activity_sessions_segments')
        activity_sessions_df = process_activity_sessions_segments(activity_sessions_df, broadcast_cal_df, geo_zip_df, geo_dma_df)

        # Add metadata
        activity_session_last_updated = pd.Timestamp.utcnow().strftime('%Y-%m-%d %H:%M:%S')
        activity_sessions_df['activity_session_last_updated'] = activity_session_last_updated
        activity_session_last_audit_id = core_functions.generate_uuid()
        activity_sessions_df['activity_session_last_audit_id'] = activity_session_last_audit_id

        # Write to GCS
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_N90_CORE_APPS')
        core_functions.write_hive_partitioned_parquet(activity_sessions_df, n90_bucket, n90_output_prefix, partition_cols, n90_storage_options)
        print(f"Finished writing to {n90_bucket}/{n90_output_prefix}")

        # Write to S4 (try-catch for error handling)
        try:
            core_functions.write_hive_partitioned_parquet_s4(activity_sessions_df, s4_bucket, s4_output_prefix, partition_cols, s4_storage_options)
            print(f"Finished writing to {s4_bucket}/{s4_output_prefix}")
        except Exception as e:
            print(f"Error writing to {s4_bucket}/{s4_output_prefix}: {e}")

        del activity_sessions_df
        gc.collect()

    except Exception as e:
        print(f"Error processing {record}: {e}")
        
# Parallel processing
def process_all_days_concurrently(process_days, broadcast_cal_df, geo_zip_df, geo_dma_df, n90_bucket, n90_output_prefix, s4_bucket, s4_output_prefix, partition_cols, n90_storage_options, s4_storage_options):
    with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:  # Adjust workers as needed
        futures = [
            executor.submit(
                process_single_day, 
                record, broadcast_cal_df, geo_zip_df, geo_dma_df, 
                n90_bucket, n90_output_prefix, s4_bucket, s4_output_prefix, 
                partition_cols, n90_storage_options, s4_storage_options
            ) 
            for record in process_days
        ]
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Exception occurred in a worker: {e}")

# Example usage
print('Beginning to write')
partition_cols = ['_YEAR', '_MONTH', '_DAY']


n90_storage_options = None
n90_storage_options = config.get('N90_GCS_STORAGE_OPTIONS')

s4_storage_options = None
s4_storage_options = config.get('S4_STORAGE_OPTIONS')

# process_df['profile__attributes']

s4_bucket = 'n90-data-lake-stl'
s4_output_prefix = 'activity/activity_sessions'


n90_bucket = None
n90_bucket = 'n90-data-lake'

n90_output_prefix = None
n90_output_prefix = 'activity/activity_sessions'


process_all_days_concurrently(
    process_days, broadcast_cal_df, geo_zip_df, geo_dma_df, 
    n90_bucket, n90_output_prefix, s4_bucket, s4_output_prefix, 
    partition_cols, n90_storage_options, s4_storage_options
)
        





In [ ]:
full_df.loc[full_df['segments_date'].isin(['2024-11-24','2024-11-25','2024-11-26']), ['segments_date', 'activity_session_timestamp', '_YEAR', '_MONTH','_WEEK_WITHIN_MONTH', 'bcw_start_datetime_match', 'bcw_end_datetime_match', 'bcw_index']]
# full_df['bcw_index'].unique()
# activity_sessions_df

In [ ]:
from datetime import datetime, timedelta

def generate_monday_midnight_bcw_start_dates(min_date, max_date):
    # Ensure inputs are datetime objects
    min_date = datetime.strptime(min_date, "%Y-%m-%d")
    max_date = datetime.strptime(max_date, "%Y-%m-%d")
    
    # Start from the Monday of the week containing min_date
    current_date = min_date - timedelta(days=min_date.weekday())
    
    # Generate list of Mondays as strings
    monday_midnights = []
    while current_date <= max_date:
        monday_midnights.append(current_date.strftime("%Y-%m-%d"))  # Convert to string
        current_date += timedelta(weeks=1)  # Increment by 1 week
    
    return monday_midnights

def generate_bcw_df(min_date, max_date, broadcast_cal_df):
    # Ensure inputs are datetime objects
    min_date = datetime.strptime(min_date, "%Y-%m-%d")
    max_date = datetime.strptime(max_date, "%Y-%m-%d")
    bcw_df = None
    
    # Start from the Monday of the week containing min_date
    current_date = min_date - timedelta(days=min_date.weekday())
    
    # Generate list of Mondays as strings
    monday_midnights = []
    while current_date <= max_date:
        monday_midnights.append(current_date.strftime("%Y-%m-%d"))  # Convert to string
        current_date += timedelta(weeks=1)  # Increment by 1 week
    for date in monday_midnights:
        print(date)
        bcw_df = pd.concat([bcw_df, broadcast_cal_df.loc[broadcast_cal_df['bcw_start_date'] == date]])
    
    return bcw_df

# test_df = activity_sessions_df[100000:300000]
activity_sessions_df['key'] = 1
min_date = (activity_sessions_df['created_time'].dt.date.astype('string')).min()
max_date = (activity_sessions_df['created_time'].dt.date.astype('string')).max()
bcw2_df = generate_bcw_df(min_date, max_date, broadcast_cal_df)
bcw2_df['key'] = 1
# _bcc_df['key'] = 1
activity_sessions_df['activity_date_time'] = pd.to_datetime(activity_sessions_df['created_time'])
bcw2_df['bcw_start_date'] = pd.to_datetime(bcw2_df['bcw_start_date'])
bcw2_df['bcw_end_date'] = pd.to_datetime(bcw2_df['bcw_end_date'])
# ref_df = None
ref_df = bcw2_df.loc[(bcw2_df['bcw_start_date'] >= activity_sessions_df['created_time'].min()) & (bcw2_df['bcw_end_date'] <= activity_sessions_df['created_time'].max())]
bcw2_df


In [ ]:

merged_df = None
merged_df = pd.merge(activity_sessions_df, bcw2_df, on='key').drop(['key', 'created_time'], axis=1)
merged_df['bc_year_index'] = merged_df['bcm_index'].astype(str).str[:4].astype('Int64')
merged_df['bcm_index'] = merged_df['bcm_index'].astype('Float64')
merged_df['bcw_index'] = merged_df['bcw_index'].astype('Float64')

merged_df.sort_values(by=['id'], inplace=True)
merged_df = merged_df.drop_duplicates(subset=['id'], keep='first')
# df = merged_df



# monday_dates = generate_monday_midnights(min_date, max_date)

# bcw_df = None
# for date in monday_dates:
#     print(date)  # Outputs Monday dates as strings
#     broadcast_cal_df['bcw_start_date'] = pd.to_datetime(broadcast_cal_df['bcw_start_date'])
    
#     # Filter DataFrame for the current Monday date
#     bcw_set = broadcast_cal_df.loc[broadcast_cal_df['bcw_start_date'] == date]
#     print(bcw_set)

In [ ]:
merged_df

In [ ]:
broadcast_cal_df.loc[broadcast_cal_df['bcw_start_date'] == '2024-10-28']
monday_dates

In [ ]:
# 27331840
# 27331844
# 27331840
activity_sessions_df.sort_values(by=['id', 'created_time'], ascending=[True, False] ,inplace=True)
activity_sessions_df = activity_sessions_df.drop_duplicates(subset=['id'], keep='first')
activity_sessions_df

In [ ]:
activity_sessions_df['_YEAR'] = activity_sessions_df['created_time'].dt.year.astype('Int64')
activity_sessions_df['_MONTH'] = activity_sessions_df['created_time'].dt.month.astype('Int64')
activity_sessions_df['_DAY'] = activity_sessions_df['created_time'].dt.day.astype('Int64')

In [ ]:
activity_sessions_with_geos_df = activity_sessions_df.merge(geo_zip_df, how='left', left_on='zip_code', right_on='geo_location')
activity_sessions_with_geos_df

In [ ]:
activity_sessions_with_geos_df.loc[activity_sessions_with_geos_df['neustar_country'].isin(['us', 'ca']) & activity_sessions_with_geos_df['geo_country'].isnull()]
activity_sessions_with_geos_df['neustar_dma_id'] = activity_sessions_with_geos_df['neustar_dma_id'].astype('string')
activity_sessions_with_geos_df.loc[activity_sessions_with_geos_df['neustar_country'].isin(['us', 'ca']) & activity_sessions_with_geos_df['geo_country'].isnull()] = activity_sessions_with_geos_df.loc[activity_sessions_with_geos_df['neustar_country'].isin(['us', 'ca']) & activity_sessions_with_geos_df['geo_country'].isnull()].merge(geo_dma_df, how='left', left_on='neustar_dma_id', right_on='geo_location')
activity_sessions_with_geos_df['neustar_dma_id'] = activity_sessions_with_geos_df['neustar_dma_id'].astype('Int64')
activity_sessions_with_geos_df.loc[activity_sessions_with_geos_df['neustar_country'].isin(['us', 'ca']) & activity_sessions_with_geos_df['geo_country'].isnull()]

In [ ]:
activity_sessions_with_geos_df['neustar_dma_id'] = activity_sessions_with_geos_df['neustar_dma_id'].astype('Int64')

In [ ]:
activity_sessions_df['created_time_2'] = pd.to_datetime(activity_sessions_df['created_time'])
broadcast_cal_df['bcw_start_date_2'] = pd.to_datetime(broadcast_cal_df['bcw_start_date'])
broadcast_cal_df['bcw_end_date_2'] = pd.to_datetime(broadcast_cal_df['bcw_end_date'])


activity_sessions_df['created_time'].max()
activity_sessions_df['_YEAR'] = activity_sessions_df['created_time'].dt.year.astype('string')
activity_sessions_df['_MONTH'] = activity_sessions_df['created_time'].dt.month.astype('string')

ref_1_ids_df = broadcast_cal_df[broadcast_cal_df['bcw_start_date_2'] <= activity_sessions_df['created_time_2'].min()]
ref_2_ids_df = broadcast_cal_df[broadcast_cal_df['bcw_end_date_2'] > activity_sessions_df['created_time_2'].max()]
ref_1_ids_df
# common_ids = []
# for id in ref_1_ids:
#     if id in ref_2_ids:
#         common_ids.append(id)
# common_ids


In [ ]:
activity_sessions_df['created_time'] = activity_sessions_df['created_time'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))



In [ ]:
activity_sessions_df.head(5)

In [ ]:
broadcast_cal_df['bcw_start_date'] = pd.to_datetime(broadcast_cal_df['bcw_start_date'])
broadcast_cal_df['bcw_end_date'] = pd.to_datetime(broadcast_cal_df['bcw_end_date'])
activity_sessions_df['created_time'] = pd.to_datetime(activity_sessions_df['created_time'])


In [ ]:
ref_min_date = pd.to_datetime(activity_sessions_df['created_time']).min()
ref_max_date = pd.to_datetime(activity_sessions_df['created_time']).max()
ref_max_date

In [ ]:

ref_1 = broadcast_cal_df[(broadcast_cal_df['bcw_start_date'] <= ref_min_date)]
# ref_2 = ref_1.loc[(ref_1['bcw_end_date'] < ref_max_date)].copy()
ref_1
# broadcast_cal_df.loc[(broadcast_cal_df['bcw_start_date'] >= activity_sessions_df['created_time'].min()) & (broadcast_cal_df['bcw_end_date'] <= activity_sessions_df['created_time'].max())]
# # Get scalar values for the date range
# min_created_time = activity_sessions_df['created_time'].min()
# max_created_time = activity_sessions_df['created_time'].max()

# # Filter the DataFrame
# filtered_broadcast_cal_df = broadcast_cal_df.loc[
#     (broadcast_cal_df['bcw_start_date'] >= min_created_time) &
#     (broadcast_cal_df['bcw_end_date'] <= max_created_time)
# ]
# filtered_broadcast_cal_df
# # reff_df
# # activity_sessions_df['created_time'].max()

In [ ]:
ddf = broadcast_cal_df
for col in ddf.columns:
    print(f'col: {col}, dtype: {ddf[col].dtype}, sample: {ddf[col].iloc[0]}')